In [60]:
from PIL import Image, ImageChops, ImageFile
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os

ImageFile.LOAD_TRUNCATED_IMAGES = True

## Check for non-padded label names and fix to 5 0's

In [63]:
for filename in os.listdir("images/labels/"):
    src = "images/labels/" + filename
    dst = filename.split('-')[-1].split('.')[0]
    if len(dst) < 5:
        dst = dst.zfill(5)
        dst = "images/labels/example-" + dst + ".png"
        os.rename(src, dst)
    


5


## Read in labels and bottle image

In [ ]:
labels_list = []
for filename in glob.glob('images/labels/*.png'): 
    im=Image.open(filename)
    labels_list.append(im)
    
bottle = Image.open('data/images/blank_bottle.jpg')

## Identify sharp color gradients and crop horizontally

In [ ]:

img_diffs = []
maxs = []
mins = []
maxs_ix = []
mins_ix = []
means = []
for label in labels_list:
    label = np.asarray(label)
    vertical_crop = label[:,:5,:]
    vertical_means = np.mean(vertical_crop, axis=2).mean(axis=1)
    diffs = []
    for i,pixel in enumerate(vertical_means):
        diff = vertical_means[i] - vertical_means[i-1]
        diffs.append(diff)
    img_diffs.append(diffs)
    max_ix = vertical_means
    #print(len(diffs)/2)
    maxs.append(np.max(diffs[256:]))
    mins.append(np.min(diffs[:256]))
    maxs_ix.append(np.argmax(diffs[256:]))
    mins_ix.append(np.argmin(diffs[:256]))
    means.append(np.mean(diffs))
    
maxs = pd.Series(maxs)
mins = pd.Series(mins)
means = pd.Series(means)
img_diffs = pd.Series(img_diffs)
    
means_good = [np.mean(diffs) for diffs in img_diffs_good]
plt.figure(figsize=(40,5))
plt.title("Average differences among labels")
plt.plot(means_good)
plt.show()

In [ ]:
for index in tqdm(img_diffs_good.index.values):
    label = labels_list[index]
    label = np.asarray(label)
    label_cropped = label[mins_ix[index]:256+maxs_ix[index],:,:]
    #print("dim:",np.shape(label_cropped))
    label_cropped = Image.fromarray(label_cropped, 'RGB')
    label_cropped.save("images/labels_cropped/cropped_" + str(index).zfill(5) + ".png", "png")
    

## Trim any remaining true whitespace

In [61]:
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

for filename in tqdm(glob.glob('images/labels_cropped/*.png')):
    im = Image.open(filename, 'r')
    im_cropped = trim(im)
    filename = filename.split('\\')[-1]
    im_cropped.save("images/labels_cropped_2/" + filename)

## Perform final quality checks

In [73]:
for path in glob.glob('images/labels_cropped_2/*.png'): 
    filename = path.split('\\')[-1].split('.')[0]
    im=Image.open(path)
    height = np.shape(im)[0]
    width = np.shape(im)[1]
    if 0.3 < (height/width) < 1.2:
        im.save("images/labels_final/" + filename + ".png")
    
    

# Place finished labels onto the bottle

In [ ]:
from PIL import ImageDraw
from tqdm import tqdm

SIZE = int(512*.58),int(512*.58)


labels_list = []
for filename in glob.glob('images/labels_final/*.png'): 
    im=Image.open(filename)
    labels_list.append(im)
    
i = 0
for current_label in tqdm(labels_list):
    # Open images
    label = current_label.copy()
    bottle = Image.open('data/images/blank_bottle.jpg')

    # Convert and resize as needed
    label = label.convert('RGBA').resize(SIZE, Image.ANTIALIAS)
    bottle = bottle.convert('RGBA')

    # Create transparency mask
    #mask=Image.new('L', label.size, color=255)
    #draw=ImageDraw.Draw(mask) 
    #label.putalpha(mask)

    bottle.paste(label, (252, 410), label)
    #bottle.show()
    #break
    bottle.save("images/labels_on_bottle/wine_bottle_" + str(i).zfill(5) + ".png", "png")
    i += 1
